![SolidQ](https://www.solidq.com/wp-content/uploads/2015/06/Logo-SolidQ-Web.gif)
# Revisión Fin de Semana I



### Cargando el conjunto de datos y obteniendo información básica:
El objetivo de esta sección es que nos familiarizemos con Pandas y seamos capaces de manipular el dataset a nuestro interés.

utilizaremos para este caso el fichero titanic_data.csv

In [0]:
import pandas
import numpy as np


input_file = 'titanic_data.csv'
separador = ","
dataset = pandas.read_csv(filepath_or_buffer=input_file, sep=separador)

# Creando el Modelo

En esta seción vamos a trabajar en la creación de un modelo que sirva para determinar si un pasajero va a sobrevivir o no. Lo primero que haremos será crear nuestro propio modelo en función de las conclusiones que hemos sacado, y después lo compararemos con un árbol de decision que entrenaremos con el dataset.



In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [0]:
# 'Survived' será nuestra etiqueta y el valor que queremos predecir:
y = dataset['Survived']
X = dataset.drop(['Survived','Name','Ticket','Cabin','PassengerId','SibSp','Parch'], axis = 1)

In [0]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.3, random_state = 0,stratify=y)

In [0]:
#Vamos a definir un Pipeline que nos permita realizar las tareas de limpieza y transformación sobre varias columnas

#Seleccionamos las caolumnas numéricas
numeric_features = ['Age', 'Fare']
#Definimos las transformaciones que les vamos a aplicar --> Imputar nulos y escalar
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

#Definimos las variables categóricas y las trnasformaciones --> Imputar con 'missing' y aplicar una Codificación OneHot

categorical_features = ['Embarked', 'Sex', 'Pclass']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

#Utilizamos el método ColumnTransformer para asociar los pipelines a las columnas

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Agregamos un clasificador de Regresión Logística
# Ahora tenemos el Pipeline completo

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', DecisionTreeClassifier(max_depth=4))])




clf.fit(X_train, y_train)

#OJO, aquí no estamos cambiando nuestro dataset original. Simplemente lanzamos este pipeline que realiza las transformaciones "al vuelo"

In [0]:


# Crear predicciones
predictions = clf.predict(X_test)

predictions

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))